The purpose of this notebook is to guide the user through use of the src package. The package is intended to be run in the command line, without modification of intermediate output. Some of the script-generated objects (such as the trained emulators and MCMC chains) can be accessed in other Python scripts or Jupyter notebooks - the details are below. Often, the most user input will come in specifying objects in the \__init\__ file.

The remainder of the document will provide the bash commands to execute the scripts of src. Note that for bash commands to be run in Jupyter Notebook, they must be proceeded by "!" - when running the commands in Terminal, leave out the exclamation point.

## Setup - \__init\__ file

The majority of the work for the user comes in setting up the \__init\__.py file. This script is called in all other scripts, and this is where the user inputs his or her own data. 

This script need not be called by the user, but it must be edited by the user. Listed below are the objects which must be changed along with a short description, though the user is encouraged to read the documentation for specifics.

* _systems_ - List of strings containing the collision systems involved. See the documentation for specifics
* _keys_ - List of strings containing the input parameters
* _labels_ - List of strings containing the LaTeX labels for the input parameters
* _ranges_ - List of tuples containing the minimum and maximum for each input parameter
* *design_array* - numpy array containing the design. If default of None remains unchanged, the code will generate a design from a Latin Hypercube
* *data_list* - Dictionary of computer model output. Should be of the form \[collision_system\]\[observable\]\[subobservable\]\['Y':, 'x':,\]. This __must__ be changed from None.
  * 'Y' is the 2D numpy array of output with rows corresponding to the rows of design_array
  * 'x' is the 1D numpy array of indexing values of the columns of 'Y'
* *exp_data_list* - Dictionary of experimental data. Should be of the form \[collision_system\]\[observable\]\[subobservable\]\[{'y':,'x:',yerr: {'stat':,'sys':}}\].  Thus __must__ be changed from None.
  * 'y' is the 1D numpy array of experimental output 
  * 'x' is the 1D numpy array of indexing values of the columns of 'y'
  * 'yerr' is a dictionary with keys 'stat' and 'sys'
  * 'stat' is a 1D numpy array of statistical errors of experimental data
  * 'sys' is a 1D numpy array of systematic errors of experimental data.
  * This must be changed from None.
* *exp_cov* - 2D numpy array of experimental covariance matrix. Recommended specified by user.
  * If left unspecified (default), it will be calculated in _mcmc.py_ as follows:
    * Each block between observables will be independent (0 matrix)
    * Within an observable, blocks of subobservables will be indepdendent unless specified in _mcmc.py_
    * Within a subobservable, the diagonal will be the sum of statistical and systematic error, and the $ij$th off-diagonal will be covariance calculated from distance between $x_i$ and $x_j$ using a squared-exponential covariance function. 
* *observables* - List of 2-tuples containing observable/subobservable pairs

## Building the Emulators - the emulator module

With design and data specified in the \__init\__ file, training the emulators is very simple. Simply run the following command:

In [ ]:
! python -m src.emulator

To change the number of principal components from the default of 10, add the --npc flag. For example, to train the emulators on 3 components, run the following command:

In [ ]:
! python -m src.emulator --npc 3

The user can also control number of restarts in the optimizer that estimates the GP hyperpameters. This is done with the --nrestarts flag.

__Important__: Once the emulators have been trained, they will be cached. After being cached, a call from the above lines will only print summaries to the console, and will not retrain the emulators. To retrain the emulators, either deleted the cached emulators or use the --retrain flag:

In [ ]:
! python -m src.emulator --retrain

### Accessing trained emulators in Jupyter

The user may wish to access the emulator outside of the scripts in this tutorial. Practical reasons include prediction or sampling for validation methods, or use in a more specialized analysis. To access the cached emulator with system 'PbPb5020' (for example), run the following lines in a python environment:

In [ ]:
from src import emulator
em = emulator.Emulator('PbPb5020')

The object _em_ will now have all functionality of the Emulator class, trained on the data specified in the \__init\__ file

## Performing Calibration - the mcmc module

This modules runs the MCMC scheme to calibrate the inputs to the experimental data, using the trained emulators as a statistical surrogate for the expensive computer model. The script calls the python distribution emcee, which runs an affine-invariant sampler. The sampler requires a number of "walkers" which each run a chain in parallel, as well as a number of iterations to run each walker. A number of burn-in steps to discarded must also be specified (these allow the samplers to "warm up"). To run this module with 500 walkers, 200 burn-in steps, and 300 post-burn-in steps, run the following command:

In [ ]:
! python -m src.mcmc --nwalkers 500 --nburnsteps 200 300

To run an additional (for example) 100 steps with the same walkers, simply remove the --nwalkers and --nburnsteps flags:

In [ ]:
! python -m src.mcmc 100

__Note__: To restart the chain, you must delete the chain.hdf file in the _mcmc/_ directory.

### Accessing posterior samples in Jupyter

The user may wish to access the posterior samples outside of the scripts in this tutorial. Practical reasons include making specific plots, or for getting posterior estimates of functions of the parameters. To access a saved chain, run the following commands in a python environment:

In [ ]:
from src import mcmc
chain = mcmc.Chain()
posterior_samples = chain.load()

The object _chain_ is an instance of the Chain class, while *posterior_samples* is a 2D numpy array where each row is a draw from the joint posterior distribution. For marginal posteriors, simply use single columns of *posterior_samples*.

## Plotting Results - the plots module

This package contains some basic plotting tools to visualize different aspects of the analysis. Plots are saved in the _plot_ directory. To create a plot, simply the add it as a positional argument. For example, to make the "posterior" plot, run the following command:

In [ ]:
! python -m src.plots posterior

To see the names of plots available for plotting, run

In [ ]:
! python -m src.plots --help

The available plots are described below:

* __observable_design__
    * Model observables at design points, with experimental data plotted as reference.
    * __IMPORTANT__: For different observables than the example, change the dictionary in \_observables_plot()
* __observable_posterior__
    * Model observables at 100 draws from the posterior, with experimental data plotted as reference.
    * __IMPORTANT__: For different observables than the example, change the dictionary in \_observables_plot()
* __posterior__
    * Pairplot of posteriors for all calibration inputs. Diagonal displays marginal density.Lower off-diagonal displays pairwise scatter plot.
* __design__
    * Projection of a LH design into two dimensions. Change keys within the function to the two inputs you want to protect into.
* __gp__
    * Conditioning a Gaussian process. Simple example plots with dummy data.
* __diag_emu__
    * Diagnostic: plots of each principal component vs each input parameter, overlaid by emulator predictions at several points in design space.
    * See how well the emulators track the design points, if uncertainty and shape of predictions are reasonable.

__Note__: The user may observe that _plots.py_ contains additional plotting functions. Some of these are helper functions, but many are copied over from the original distribution from which this distribution is forked. Most of the unreported functions are hard-coded for that project's observables, so support for those functions were excluded in this package.